# Traditional Data Engineering - ETL Pipeline

This notebook demonstrates traditional ETL operations using Spark and Delta Lake for customer support ticket data.

## Overview
- Extract CSV data from S3
- Transform data format
- Load into Delta Lake format
- Create external tables in Unity Catalog

## 1. Explore Available Catalogs

Check what catalogs are available in the Unity Catalog environment.

In [0]:
%sql
SHOW CATALOGS;

catalog
hive_metastore
samples
system
workspace


## 2. ETL Process - Extract, Transform, Load

Process customer support tickets from CSV to Delta format with external table registration.

In [0]:
# Define the path to your external Delta table location
bronze_table_path = "s3://awsdbjuly/bronze/bronze_tickets"

# Step 1: Read CSV
support_tickets_path = "s3://awsdbjuly/support_tickets.csv"
df = (spark.read
      .option("header", "true")
      .option("inferSchema", "true")
      .csv(support_tickets_path))

# Step 2: Write to external S3 path in Delta format (not as a managed table)
(df.write
   .format("delta")
   .mode("overwrite")
   .save(bronze_table_path))

# Step 3 (optional): Register as external table in Unity Catalog
# Replace 'workspace' with your catalog and 'smart_support' with your schema
spark.sql("CREATE SCHEMA IF NOT EXISTS workspace.smart_support")
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS workspace.smart_support.bronze_tickets
    USING DELTA
    LOCATION '{bronze_table_path}'
""")

# Verify the table
display(spark.table("workspace.smart_support.bronze_tickets"))
